In [1]:
import numpy as np
import random as rm
import pandas as pd
from random import seed
from random import random
import matplotlib.pyplot as plt
from pylab import *
from scipy.optimize import curve_fit
from scipy.stats import bernoulli, binom, norm
from scipy import integrate

In [2]:
def simulate(mu_list, sigma_list, alpha_list, n, p, x0_list, y0_list, sigma, epsilon,N):
    def f_pi(x):
        val = 0
        for i in range(len(mu_list)):
            val += alpha_list[i]*norm.pdf(x,mu_list[i], sigma_list[i]**0.5)
        return val

    def pdf_eta(x):
        return f_pi(x)**p

    c = integrate.quad(pdf_eta, -np.inf, np.inf)[0]
    
    def f_eta(x):
        return pdf_eta(x)/c
    
    def g(x):
        return(f_pi(x)/f_eta(x))
    
    
    # adjusted version of p_lernel
    def p_kernel_adjust(y0_list, sigma, N):
        Yn_list = y0_list
        estimator_matrix = np.empty((0,N+1), int)
        for i in range(len(Yn_list)):
            count = 0
            estimator_list = [y0_list[i]]
            Yn_list = y0_list
            while count<N:
                y = (np.random.normal(Yn_list[i], sigma))
                u = np.random.uniform(0,1)
                alpha = (min((pdf_eta(y)/pdf_eta(Yn_list[i])),1))
                if u <= alpha:
                    estimator_list.append(y)
                    Yn_list[i] = y
                    count += 1
            estimator_matrix = np.append(estimator_matrix, [estimator_list], axis=0)
        return estimator_matrix

    # adjusted version of k_kernel
    def k_kernel_adjust(x0_list,sigma):
        Xn_list = x0_list
        estimator_matrix = np.empty((0,1), int)
        for i in range(len(Xn_list)):
            count = 0
            estimator_list = [x0_list[i]]
            Xn_list = x0_list
            while count<1:
                y = np.random.normal(Xn_list[i],sigma)
                u = np.random.uniform(0,1)
                alpha = min((pdf_eta(y)/pdf_eta(Xn_list[i])),1)
                if u <= alpha:
                    estimator_list.append(y)
                    Xn_list[i] = y
                    count +=1
            estimator_matrix = np.append(estimator_matrix, [estimator_list[1:]], axis=0)
        return estimator_matrix
    
    def sum_g(sum_old, yn):
        sum_new = sum_old + g(yn)
        return sum_new

    def prob_list(y, sum_new): 
        prob_list = []
        for i in y:
            prob_list.append(g(i)/sum_new)
        return prob_list
    
    def bernoulli_list(epsilon, n):
        b_list = bernoulli(epsilon).rvs(n)
        return b_list
    
    # new function (transform x0_list into a matrix)
    def X_list_dec (x0_list, N):
        new_matrix = np.empty((0,1), int)
        for i in range(N):
            new_matrix = np.append(new_matrix, [[x0_list[i]]], axis = 0)
        return new_matrix
    
    # adjusted version of x_sample_new
    def x_sample_adjust(x0_list, epsilon, y0_list, n, sigma,N):
        X_list = X_list_dec (x0_list, N)
        X_list_new = np.empty((0,2), int)
        #X_list = np.array([x0_list])
        #X_list = np.empty((0,1), int)
        Y_list = np.array([y0_list])
        Y_list = p_kernel_adjust(y0_list,sigma,N)
        for i in range(N):
            sum_gs = g(y0_list[i])
            sub_list = list(X_list[i])

        #sum_gs = g(y0_list[0])
        #tm = list(X_list[0])
            for k in range(1,n):
                sum_gs = sum_g(sum_gs, Y_list[i][k])
                prob_list_Ys = prob_list(Y_list[i][:k+1], sum_gs)
                bernoullis = bernoulli_list(epsilon, 1)
                i = bernoullis
                if i == 0:
                    #X_list.append(k_kernel_adjust(x0_list[i], sigma))
                    sub_list.append(int(k_kernel_adjust(x0_list, sigma)[i][0]))
        
                if i == 1:
                    #X_list.append(np.random.choice(Y_list[i][:k+1], p = prob_list_Ys))
                    sub_list.append(int(np.random.choice(int(Y_list[i][:k+1]), p = prob_list_Ys)))
                    x0_list[i] = sub_list[-1]
            X_list_new = np.append(X_list_new,[sub_list],axis=0)
        return X_list_new
    
    #X_new = x_sample_new(x0_list, epsilon, y0_list, n, sigma, N)
    #return X_new
    
    # adjusted version of E_x
    def E_x(X_list):
        sum_matrix = X_list.sum()
        len_matrix = X_list.size
        average_matrix = sum_matrix/len_matrix
        return average_matrix
        #mean = sum/len(X_list)
        #return mean
    
    X_list = x_sample_adjust(x0_list, epsilon, y0_list, n, sigma,N)
    X_mean = E_x(X_list)
    
    return [X_list, X_mean]
    

In [3]:
mu_list = [0,17.5]        # mean list of target distribution
sigma_list = [.5,1]       # variance list of target distribtuion
alpha_list = [0.4,0.6]    # constant list of target distribution
p = 0.75                  # the rate coefficient between target distribution and proposed distribution
n = 2               # number of simulations
y0_list = [15,13,14,15,16,18]                # initial position of y
x0_list = [10,-6,8,15,4,7]                  # initial position of x
#y0_list = [15,14]
#x0_list = [10,12]
sigma = 10                # 
epsilon = 0.05            #
N= len(x0_list)

In [4]:
simulate(mu_list, sigma_list, alpha_list, n, p, x0_list, y0_list, sigma, epsilon,N)


[array([[10, 13],
        [-6, 17],
        [ 8,  1],
        [15,  0],
        [ 4, 17],
        [ 7, 16]]),
 8.5]